# Market Simulator
ML for trading Udacity Course exercise

More info:
http://wiki.quantsoftware.org/index.php?title=CompInvesti_Homework_3

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairoart 2018
"""


## Overview

In this project you will create a basic market simulator that accepts trading orders and keeps track of a portfolio's value and saves it to a file. You will also create another program that assesses the performance of that portfolio. 

## Part 1: Create a market simulation tool

Starting cash: 1000000  
Input file: orders.csv 
Output file: values.csv

The file of orders is organized like this:

    Year
    Month
    Day
    Symbol
    BUY or SELL
    Number of Shares 
    
For example:

    2008, 12, 3, AAPL, BUY, 130
    2008, 12, 8, AAPL, SELL, 130
    2008, 12, 5, IBM, BUY, 50


### Goal

Your simulator should calculate the total value of the portfolio for each day using adjusted closing prices (cash plus value of equities) and print the result to the file values.csv. The contents of the values.csv file should look something like this:

    2008, 12, 3, 1000000
    2008, 12, 4, 1000010
    2008, 12, 5, 1000250
    ...


### Import libraries

In [132]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import operator
import sys
import csv

# To fetch data
from pandas_datareader import data as pdr   
import fix_yahoo_finance as yf  
yf.pdr_override()   

### Read orders from file

In [133]:
def readOrdersFileIntoDF(filename):

    # opening the filename
    fr = open(filename)
    
    # for row count in 
    index=0
    
    # Lists used for making the dataframe.
    dtList = []
    symbolList = []
    orderTypeList = []
    volumeList = []
    
    # For each line
    # A Sample Line - 2011,1,14,AAPL,Buy,1500
    for orderString in fr.readlines():

        # Stripping off the return line character
        orderString=orderString.strip()
        
        # Splitting the line and getting a List back
        listFromLine = orderString.split(',')
        
        # Adding the dates into dtList. 16,00,00 for 1600 hrs
        dtList.append(dt.datetime(int(listFromLine[0]), int(listFromLine[1]), int(listFromLine[2])))
        
        # Adding the symbols into symbolList
        symbolList.append(listFromLine[3])
        
        # Adding the orders into orderTypeList
        orderTypeList.append(listFromLine[4])
        
        # Adding the number of shares into volumeList
        volumeList.append(listFromLine[5])

    # Creating a Dictionary for converting it into DataFrame later
    data = { 'datetime' : dtList, 'symbol' : symbolList, 'ordertype':orderTypeList, 'volume':volumeList }

    # Converting the Dictinary into a nice looking Pandas Dataframe ordered by datetime index
    ordersDataFrame = pd.DataFrame(data)
    ordersDataFrame.index= ordersDataFrame['datetime']
    
    # Drop datetime column
    ordersDataFrame.drop('datetime', axis=1, inplace=True)
    #print(ordersDataFrame)
    
    
    # Getting the Symbols from the Orders. This list will be required for fetching the prices
    symbolList = list(set(ordersDataFrame['symbol']))
    
    # Returning it.
    return ordersDataFrame, symbolList

### Get data from Yahoo for the dates 

In [134]:
def fetchData(dt_start, dt_end, ls_symbols):
    
    # Get data of trading days between the start and the end.
    df = pdr.get_data_yahoo(ls_symbols, dt_start, dt_end)       
    nan_rows = df.isnull().sum().sum()
    #df.info()
    #print(nan_rows)

    # Getting the numpy ndarray of close prices.
    na_price = df['Close'].values

    # returning the closed prices for all the days    
    return na_price, df

### Market simulator

In [135]:
def marketsim(initialCash, ordersdf, symbols):

    # reading the boundary dates
    dt_start = ordersdf.index[0]
    dt_end = ordersdf.index[len(ordersdf)-1]

    # All the adjustedClosingPrices fetched from NYSE within the range and for given symbols
    closingPrices, ldt_timestamps = fetchData(dt_start, dt_end, symbols)

    num_tradingDays = len(ldt_timestamps)
 
    # For Holdings of the share
    temp = np.zeros((1, len(symbols)))
    holdings = pd.DataFrame(temp, columns = symbols, index = ['holdings'])
    
    #Cash for the days
    temp = np.zeros((num_tradingDays, 1))
    cash = pd.DataFrame(temp, columns = ['cashinhand'])
    
    #Value for the days
    temp = np.zeros((num_tradingDays, 1))
    valueFrame = pd.DataFrame(temp, columns = ['valueOfPortfolio'])

    #Setting the first value to be the initial cash amount.
    cash.cashinhand.loc[0] = initialCash
    
    index = 0
    
    for tradingDayIndex in range(num_tradingDays):
        if tradingDayIndex != 0:
            cash.cashinhand.loc[tradingDayIndex] = cash.cashinhand.loc[tradingDayIndex - 1] 
        else:
            cash.cashinhand.loc[tradingDayIndex] = initialCash
      
   
        for tradingOrder in ordersDataFrame.index:
            if tradingOrder == ldt_timestamps.index[tradingDayIndex]:
                if ordersdf.ordertype.iloc[index] == 'Buy':
                    toBuySymbol = ordersdf.symbol.iloc[index]
                    toBuy = symbols.index(toBuySymbol)
                    numShares = ordersdf.volume.iloc[index]
                    priceForTheDay = closingPrices[tradingDayIndex, toBuy]
                    cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex] - (priceForTheDay * float(numShares))
                    holdings[toBuySymbol].iloc[0] += int(numShares)

                elif ordersdf.ordertype.iloc[index] == 'Sell':
                    toSellSymbol = ordersdf.symbol.iloc[index]
                    toSell = symbols.index(toSellSymbol)
                    numShares = ordersdf.volume.iloc[index]
                    priceForTheDay = closingPrices[tradingDayIndex, toSell]
                    cash.cashinhand.iloc[tradingDayIndex] = cash.cashinhand.iloc[tradingDayIndex] + (priceForTheDay * float(numShares))
                    holdings[toSellSymbol].iloc[0] -= int(numShares)
                else:
                    print("error")
                index+=1

        valueFromPortfolio = 0

        for symbol in symbols:
            priceForTheDay = closingPrices[tradingDayIndex, symbols.index(symbol)]
            valueFromPortfolio += holdings[symbol].iloc[0] * priceForTheDay

        valueFrame.valueOfPortfolio.iloc[tradingDayIndex] = valueFromPortfolio + cash.cashinhand.iloc[tradingDayIndex]

    valueFrame.index = ldt_timestamps.index
    return holdings, valueFrame, cash

### Write values to file

In [ ]:
def writeValuesIntoCSV(valuesFilename, valueFrame):
    file = open(valuesFilename, 'w')
    writer = csv.writer(file)
    
    for index in range(len(valueFrame)):
        writer.writerow([valueFrame.index[index].year, valueFrame.index[index].month, valueFrame.index[index].day ,int(round(valueFrame.valueOfPortfolio.iloc[index], 0))])
    
    file.close()

### Define initial values and execute simulation

In [ ]:
initialCash = 1000000
ordersFilename = 'input/orders.csv'
valuesFilename = 'output/values.csv'

# Reading the data from the file, and getting a NumPy matrix
ordersDataFrame, symbols = readOrdersFileIntoDF(ordersFilename)

holdings, valueFrame, cash = marketsim(initialCash, ordersDataFrame, symbols)

writeValuesIntoCSV(valuesFilename, valueFrame)

[                       0%                       ]